# StateFarm Distracted Driver Detection

In [35]:
%cd /home/ubuntu/kaggle/state-farm-distracted-driver-detection
# Make sure you are in the main directory (state-farm-distracted-driver-detection)
%pwd

/home/ubuntu/kaggle/state-farm-distracted-driver-detection


u'/home/ubuntu/kaggle/state-farm-distracted-driver-detection'

In [36]:
# Create references to key directories
import os, sys
from glob import glob
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
current_dir = os.getcwd()
CHALLENGE_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data'

In [5]:
#Allow relative imports to directories
sys.path.insert(1, os.path.join(sys.path[0], '..'))

#import modules
from utils import *
from utils.vgg16 import Vgg16

#Instantiate plotting tool
%matplotlib inline

## Create Validation and Sample Sets

In [6]:
#Create directories
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data


In [7]:
# Create folders for valid/
%cd $DATA_HOME_DIR/valid/
!for i in `seq 0 9`; do mkdir "c"$i; done

# Create folders for sample/train/
%cd $DATA_HOME_DIR/sample/train
!for i in `seq 0 9`; do mkdir "c"$i; done

#Create folders for sample/valid
%cd $DATA_HOME_DIR/sample/valid
!for i in `seq 0 9`; do mkdir "c"$i; done

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data/valid


In [37]:
# Move images to validation sets
%cd $DATA_HOME_DIR

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data


In [39]:
for i in range(10):
    directory = DATA_HOME_DIR + '/train/c'+ str(i)
    l = os.listdir(directory)
    shuf = np.random.permutation(l)
    for j in range(1000): os.rename(directory+'/'+shuf[j], DATA_HOME_DIR+'/valid/c'+str(i) + '/' + shuf[j])

In [40]:
from shutil import copyfile

In [41]:
# Copy images from train to sample/train and sample/valid sets
%cd $DATA_HOME_DIR

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data


In [44]:
for i in range(10):
    directory = DATA_HOME_DIR + '/train/c'+ str(i)
    l = os.listdir(directory)
    shuf = np.random.permutation(l)
    for j in range(100): copyfile(directory+'/'+shuf[j], DATA_HOME_DIR+'/sample/train/c'+str(i) + '/' + shuf[j])

In [45]:
# Copy images from valid to sample/train and sample/valid sets
for i in range(10):
    directory = DATA_HOME_DIR + '/valid/c'+ str(i)
    l = os.listdir(directory)
    shuf = np.random.permutation(l)
    for j in range(100): copyfile(directory+'/'+shuf[j], DATA_HOME_DIR+'/sample/valid/c'+str(i) + '/' + shuf[j])

In [57]:
# Move a sample of test to sample/test

directory = DATA_HOME_DIR + '/test/'
l = os.listdir(directory)
shuf = np.random.permutation(l)
for j in range(1000): copyfile(directory+'/'+shuf[j], DATA_HOME_DIR+'/sample/test/'+ shuf[j])

## Create a Unkown class for the test set

In [46]:
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data/test


## Finetuning and Training

In [63]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/sample'
test_path = path + '/test/' #We use all the test data
results_path= path + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data


In [64]:
#import Vgg16 helper class
vgg = Vgg16()

In [68]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=20

In [66]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

vgg.model.optimizer.lr = 0.01

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [69]:
#Pass in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

Running epoch: 0
Epoch 1/1
1000/1000 [==============================] - 53s - loss: 10.7241 - acc: 0.2530 - val_loss: 9.9149 - val_acc: 0.3180
Running epoch: 1
Epoch 1/1
1000/1000 [==============================] - 53s - loss: 10.9520 - acc: 0.2520 - val_loss: 10.1562 - val_acc: 0.3060
Running epoch: 2
Epoch 1/1
1000/1000 [==============================] - 54s - loss: 11.0160 - acc: 0.2520 - val_loss: 10.3218 - val_acc: 0.3080
Running epoch: 3
Epoch 1/1
1000/1000 [==============================] - 54s - loss: 10.7834 - acc: 0.2750 - val_loss: 10.1912 - val_acc: 0.2990
Running epoch: 4
Epoch 1/1
1000/1000 [==============================] - 53s - loss: 11.0091 - acc: 0.2570 - val_loss: 10.3288 - val_acc: 0.2970
Running epoch: 5
Epoch 1/1
1000/1000 [==============================] - 53s - loss: 10.7376 - acc: 0.2870 - val_loss: 10.2916 - val_acc: 0.3020
Running epoch: 6
Epoch 1/1
1000/1000 [==============================] - 53s - loss: 10.9899 - acc: 0.2750 - val_loss: 10.4523 - val_acc: 

## Generate Predictions

Make predictions on the test data!

In [75]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

Found 0 images belonging to 0 classes.


Exception in thread Thread-108:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 425, in data_generator_task
    generator_output = next(generator)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 593, in next
    index_array, current_index, current_batch_size = next(self.index_generator)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 441, in _flow_index
    current_index = (self.batch_index * batch_size) % N
ZeroDivisionError: integer division or modulo by zero



In [74]:
test_path

'/home/ubuntu/kaggle/state-farm-distracted-driver-detection/data/sample/test/'

In [ ]:
#For every image, vgg.test() generates two probabilities 
#based on how we've ordered the train directories.
print preds[:5]

filenames = batches.filenames
print filenames[:5]

In [ ]:
#Verify the column ordering by viewing some images
from PIL import Image
Image.open(test_path + filenames[2])

In [ ]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)